In [1]:
# author - Richard Liao 
# Dec 26 2016
import numpy as np
import pandas as pd

from collections import defaultdict
import re

from bs4 import BeautifulSoup

import sys
import os

os.environ['KERAS_BACKEND']='tensorflow'

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

#from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten, Lambda, Concatenate, Reshape
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM, GRU, Bidirectional, TimeDistributed
from keras.models import Model

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session





Using TensorFlow backend.


In [2]:
def gpu_alloc(device_id):
    os.environ["CUDA_VISIBLE_DEVICES"]=device_id
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    set_session(tf.Session(config=config))

In [ ]:
gpu_alloc("1")

# Data


In [4]:
MAX_SENT_LENGTH = 100
MAX_SENTS = 15
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2
NUM_EPOCHS = 100
BATCH_SIZE = 50

def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\\", "", string)    
    string = re.sub(r"\'", "", string)    
    string = re.sub(r"\"", "", string)    
    return string.strip().lower()

data_train = pd.read_csv('./dat/imdb/labeledTrainData.tsv', sep='\t')
print(data_train.shape)

from nltk import tokenize

reviews = []
labels = []
texts = []

for idx in range(data_train.review.shape[0]):
    print('Parsing review ' + str(idx))
    text = BeautifulSoup(data_train.review[idx]).get_text()
    text = clean_str(text)#.get_text().encode('ascii','ignore'))
    #print('Text:\n' + text)
    texts.append(text)
    sentences = tokenize.sent_tokenize(text)
    reviews.append(sentences)
    '''
    for sent in sentences:
          print('Sentence:\n' + sent)
    '''
    labels.append(data_train.sentiment[idx])

tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)

data = np.zeros((len(texts), MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')

for i, sentences in enumerate(reviews):
    for j, sent in enumerate(sentences):	
        if j< MAX_SENTS:
            wordTokens = text_to_word_sequence(sent)
            k=0
            for _, word in enumerate(wordTokens):
                if k<MAX_SENT_LENGTH and tokenizer.word_index[word]<MAX_NB_WORDS:
                    data[i,j,k] = tokenizer.word_index[word]
                    k=k+1                    
                    
word_index = tokenizer.word_index
print('Total %s unique tokens.' % len(word_index))

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

print('Number of positive and negative reviews in traing and validation set')
print(y_train.sum(axis=0))
print(y_val.sum(axis=0))



(25000, 3)
Parsing review 0
Parsing review 1
Parsing review 2
Parsing review 3
Parsing review 4
Parsing review 5
Parsing review 6
Parsing review 7
Parsing review 8
Parsing review 9
Parsing review 10
Parsing review 11
Parsing review 12
Parsing review 13
Parsing review 14
Parsing review 15
Parsing review 16
Parsing review 17
Parsing review 18
Parsing review 19
Parsing review 20
Parsing review 21
Parsing review 22
Parsing review 23
Parsing review 24
Parsing review 25
Parsing review 26
Parsing review 27
Parsing review 28
Parsing review 29
Parsing review 30
Parsing review 31
Parsing review 32
Parsing review 33
Parsing review 34
Parsing review 35
Parsing review 36
Parsing review 37
Parsing review 38
Parsing review 39
Parsing review 40
Parsing review 41
Parsing review 42
Parsing review 43
Parsing review 44
Parsing review 45
Parsing review 46
Parsing review 47
Parsing review 48
Parsing review 49
Parsing review 50
Parsing review 51
Parsing review 52
Parsing review 53
Parsing review 54
Parsing r

/opt/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /opt/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


Parsing review 205
Parsing review 206
Parsing review 207
Parsing review 208
Parsing review 209
Parsing review 210
Parsing review 211
Parsing review 212
Parsing review 213
Parsing review 214
Parsing review 215
Parsing review 216
Parsing review 217
Parsing review 218
Parsing review 219
Parsing review 220
Parsing review 221
Parsing review 222
Parsing review 223
Parsing review 224
Parsing review 225
Parsing review 226
Parsing review 227
Parsing review 228
Parsing review 229
Parsing review 230
Parsing review 231
Parsing review 232
Parsing review 233
Parsing review 234
Parsing review 235
Parsing review 236
Parsing review 237
Parsing review 238
Parsing review 239
Parsing review 240
Parsing review 241
Parsing review 242
Parsing review 243
Parsing review 244
Parsing review 245
Parsing review 246
Parsing review 247
Parsing review 248
Parsing review 249
Parsing review 250
Parsing review 251
Parsing review 252
Parsing review 253
Parsing review 254
Parsing review 255
Parsing review 256
Parsing revi

Parsing review 814
Parsing review 815
Parsing review 816
Parsing review 817
Parsing review 818
Parsing review 819
Parsing review 820
Parsing review 821
Parsing review 822
Parsing review 823
Parsing review 824
Parsing review 825
Parsing review 826
Parsing review 827
Parsing review 828
Parsing review 829
Parsing review 830
Parsing review 831
Parsing review 832
Parsing review 833
Parsing review 834
Parsing review 835
Parsing review 836
Parsing review 837
Parsing review 838
Parsing review 839
Parsing review 840
Parsing review 841
Parsing review 842
Parsing review 843
Parsing review 844
Parsing review 845
Parsing review 846
Parsing review 847
Parsing review 848
Parsing review 849
Parsing review 850
Parsing review 851
Parsing review 852
Parsing review 853
Parsing review 854
Parsing review 855
Parsing review 856
Parsing review 857
Parsing review 858
Parsing review 859
Parsing review 860
Parsing review 861
Parsing review 862
Parsing review 863
Parsing review 864
Parsing review 865
Parsing revi

Parsing review 1413
Parsing review 1414
Parsing review 1415
Parsing review 1416
Parsing review 1417
Parsing review 1418
Parsing review 1419
Parsing review 1420
Parsing review 1421
Parsing review 1422
Parsing review 1423
Parsing review 1424
Parsing review 1425
Parsing review 1426
Parsing review 1427
Parsing review 1428
Parsing review 1429
Parsing review 1430
Parsing review 1431
Parsing review 1432
Parsing review 1433
Parsing review 1434
Parsing review 1435
Parsing review 1436
Parsing review 1437
Parsing review 1438
Parsing review 1439
Parsing review 1440
Parsing review 1441
Parsing review 1442
Parsing review 1443
Parsing review 1444
Parsing review 1445
Parsing review 1446
Parsing review 1447
Parsing review 1448
Parsing review 1449
Parsing review 1450
Parsing review 1451
Parsing review 1452
Parsing review 1453
Parsing review 1454
Parsing review 1455
Parsing review 1456
Parsing review 1457
Parsing review 1458
Parsing review 1459
Parsing review 1460
Parsing review 1461
Parsing review 1462


...

Parsing review 24122
Parsing review 24123
Parsing review 24124
Parsing review 24125
Parsing review 24126
Parsing review 24127
Parsing review 24128
Parsing review 24129
Parsing review 24130
Parsing review 24131
Parsing review 24132
Parsing review 24133
Parsing review 24134
Parsing review 24135
Parsing review 24136
Parsing review 24137
Parsing review 24138
Parsing review 24139
Parsing review 24140
Parsing review 24141
Parsing review 24142
Parsing review 24143
Parsing review 24144
Parsing review 24145
Parsing review 24146
Parsing review 24147
Parsing review 24148
Parsing review 24149
Parsing review 24150
Parsing review 24151
Parsing review 24152
Parsing review 24153
Parsing review 24154
Parsing review 24155
Parsing review 24156
Parsing review 24157
Parsing review 24158
Parsing review 24159
Parsing review 24160
Parsing review 24161
Parsing review 24162
Parsing review 24163
Parsing review 24164
Parsing review 24165
Parsing review 24166
Parsing review 24167
Parsing review 24168
Parsing revie

Parsing review 24528
Parsing review 24529
Parsing review 24530
Parsing review 24531
Parsing review 24532
Parsing review 24533
Parsing review 24534
Parsing review 24535
Parsing review 24536
Parsing review 24537
Parsing review 24538
Parsing review 24539
Parsing review 24540
Parsing review 24541
Parsing review 24542
Parsing review 24543
Parsing review 24544
Parsing review 24545
Parsing review 24546
Parsing review 24547
Parsing review 24548
Parsing review 24549
Parsing review 24550
Parsing review 24551
Parsing review 24552
Parsing review 24553
Parsing review 24554
Parsing review 24555
Parsing review 24556
Parsing review 24557
Parsing review 24558
Parsing review 24559
Parsing review 24560
Parsing review 24561
Parsing review 24562
Parsing review 24563
Parsing review 24564
Parsing review 24565
Parsing review 24566
Parsing review 24567
Parsing review 24568
Parsing review 24569
Parsing review 24570
Parsing review 24571
Parsing review 24572
Parsing review 24573
Parsing review 24574
Parsing revie

Parsing review 24940
Parsing review 24941
Parsing review 24942
Parsing review 24943
Parsing review 24944
Parsing review 24945
Parsing review 24946
Parsing review 24947
Parsing review 24948
Parsing review 24949
Parsing review 24950
Parsing review 24951
Parsing review 24952
Parsing review 24953
Parsing review 24954
Parsing review 24955
Parsing review 24956
Parsing review 24957
Parsing review 24958
Parsing review 24959
Parsing review 24960
Parsing review 24961
Parsing review 24962
Parsing review 24963
Parsing review 24964
Parsing review 24965
Parsing review 24966
Parsing review 24967
Parsing review 24968
Parsing review 24969
Parsing review 24970
Parsing review 24971
Parsing review 24972
Parsing review 24973
Parsing review 24974
Parsing review 24975
Parsing review 24976
Parsing review 24977
Parsing review 24978
Parsing review 24979
Parsing review 24980
Parsing review 24981
Parsing review 24982
Parsing review 24983
Parsing review 24984
Parsing review 24985
Parsing review 24986
Parsing revie

/home/elsallab/.local/lib/python3.6/site-packages/keras_preprocessing/text.py:177: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Total 81501 unique tokens.
Shape of data tensor: (25000, 15, 100)
Shape of label tensor: (25000, 2)
Number of positive and negative reviews in traing and validation set
[  9966.  10034.]
[ 2534.  2466.]


# Model


In [5]:

GLOVE_DIR = "./dat/glove"
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

f.close()

print('Total %s word vectors.' % len(embeddings_index))

embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

w_emb_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            #input_length=1,
                            trainable=True)
sent_enc_layer = Bidirectional(LSTM(100))
'''
w_input = Input(shape=(), dtype='int32')
w_emb = w_emb_layer(w_input)
w_emb = Reshape((1,w_emb.shape[1]))(w_emb)
w_emb_model = Model(w_input, w_emb)
'''

rev_enc_layer = Bidirectional(LSTM(100))

review_input = Input(shape=(MAX_SENTS,MAX_SENT_LENGTH), dtype='int32')

s_embs = []
for i in range(MAX_SENTS):
    sent = Lambda(lambda x: x[:,i,:])(review_input)#sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
    w_embs = []
    for j in range(MAX_SENT_LENGTH):
        word = Lambda(lambda x: x[:,j])(sent)
        
        w_emb = w_emb_layer(word)
        #print(w_emb.shape)
        #print(w_emb._keras_shape)
        w_emb = Reshape((1,int(w_emb.shape[1])))(w_emb)
        #rint(w_emb.shape[1])
        #w_emb = Reshape((1,100))(w_emb)
        #rint(w_emb._keras_shape)
        #w_emb = w_emb_model(word)
        w_embs.append(w_emb)
    
    #print(w_embs[0]._keras_shape)
    w_embs = Concatenate(axis=-2)(w_embs)
    #print(w_embs_.shape)
    s_emb = sent_enc_layer(w_embs)
    #print(s_emb.shape)
    s_emb = Reshape((1, int(s_emb.shape[1])))(s_emb)
    s_embs.append(s_emb)

s_embs = Concatenate(axis=-2)(s_embs)
rev_emb = rev_enc_layer(s_embs)

preds = Dense(2, activation='softmax')(rev_emb)
model = Model(review_input, preds)
'''
sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sentence_input)
l_lstm = Bidirectional(LSTM(100))(embedded_sequences)
sentEncoder = Model(sentence_input, l_lstm)
print(sentEncoder.summary())
review_input = Input(shape=(MAX_SENTS,MAX_SENT_LENGTH), dtype='int32')
review_encoder = TimeDistributed(sentEncoder)(review_input)
print(review_encoder.shape)
l_lstm_sent = Bidirectional(LSTM(100))(review_encoder)
preds = Dense(2, activation='softmax')(l_lstm_sent)
model = Model(review_input, preds)
'''
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])



Total 7396 word vectors.


In [6]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 15, 100)      0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 100)          0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_102 (Lambda)             (None, 100)          0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_203 (Lambda)             (None, 100)          0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_304

__________________________________________________________________________________________________
lambda_1070 (Lambda)            (None,)              0           lambda_1011[0][0]                
__________________________________________________________________________________________________
lambda_1071 (Lambda)            (None,)              0           lambda_1011[0][0]                
__________________________________________________________________________________________________
lambda_1072 (Lambda)            (None,)              0           lambda_1011[0][0]                
__________________________________________________________________________________________________
lambda_1073 (Lambda)            (None,)              0           lambda_1011[0][0]                
__________________________________________________________________________________________________
lambda_1074 (Lambda)            (None,)              0           lambda_1011[0][0]                
__________

                                                                 lambda_858[0][0]                 
                                                                 lambda_859[0][0]                 
                                                                 lambda_860[0][0]                 
                                                                 lambda_861[0][0]                 
                                                                 lambda_862[0][0]                 
                                                                 lambda_863[0][0]                 
                                                                 lambda_864[0][0]                 
                                                                 lambda_865[0][0]                 
                                                                 lambda_866[0][0]                 
                                                                 lambda_867[0][0]                 
          

reshape_933 (Reshape)           (None, 1, 100)       0           embedding_1[923][0]              
__________________________________________________________________________________________________
reshape_934 (Reshape)           (None, 1, 100)       0           embedding_1[924][0]              
__________________________________________________________________________________________________
reshape_935 (Reshape)           (None, 1, 100)       0           embedding_1[925][0]              
__________________________________________________________________________________________________
reshape_936 (Reshape)           (None, 1, 100)       0           embedding_1[926][0]              
__________________________________________________________________________________________________
reshape_937 (Reshape)           (None, 1, 100)       0           embedding_1[927][0]              
__________________________________________________________________________________________________
reshape_93

In [8]:

print("model fitting - Hierachical LSTM")

model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=NUM_EPOCHS, batch_size=BATCH_SIZE)


/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """


model fitting - Hierachical LSTM
Train on 20000 samples, validate on 5000 samples
Epoch 1/100
20000/20000 [==============================] - 1683s 84ms/step - loss: 0.5656 - acc: 0.6986 - val_loss: 0.3701 - val_acc: 0.8380
Epoch 2/100
 3950/20000 [====>.........................] - ETA: 19:10 - loss: 0.3899 - acc: 0.8299

KeyboardInterrupt: 

# Test

In [9]:

data_test = pd.read_csv('./dat/imdb/labeledTrainData.tsv', sep='\t')
print(data_test.shape)


reviews = []
texts = []

for idx in range(data_test.review.shape[0]):
    print('Parsing review ' + str(idx))
    text = BeautifulSoup(data_test.review[idx]).get_text()
    text = clean_str(text)#.get_text().encode('ascii','ignore'))

    texts.append(text)
    sentences = tokenize.sent_tokenize(text)
    reviews.append(sentences)


data = np.zeros((len(texts), MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')

for i, sentences in enumerate(reviews):
    for j, sent in enumerate(sentences):	
        if j< MAX_SENTS:
            wordTokens = text_to_word_sequence(sent)
            k=0
            for _, word in enumerate(wordTokens):
                if k<MAX_SENT_LENGTH and tokenizer.word_index[word]<MAX_NB_WORDS:
                    data[i,j,k] = tokenizer.word_index[word]
                    k=k+1                    
                    
test_input_data = data
test_texts = list(data_test.review.apply(BeautifulSoup).apply(BeautifulSoup.get_text).apply(clean_str))

(25000, 3)
Parsing review 0
Parsing review 1
Parsing review 2
Parsing review 3
Parsing review 4
Parsing review 5
Parsing review 6
Parsing review 7
Parsing review 8
Parsing review 9
Parsing review 10
Parsing review 11
Parsing review 12
Parsing review 13
Parsing review 14
Parsing review 15
Parsing review 16
Parsing review 17
Parsing review 18
Parsing review 19
Parsing review 20
Parsing review 21
Parsing review 22
Parsing review 23
Parsing review 24
Parsing review 25
Parsing review 26
Parsing review 27
Parsing review 28
Parsing review 29
Parsing review 30
Parsing review 31
Parsing review 32
Parsing review 33
Parsing review 34
Parsing review 35
Parsing review 36
Parsing review 37
Parsing review 38
Parsing review 39
Parsing review 40
Parsing review 41
Parsing review 42
Parsing review 43
Parsing review 44
Parsing review 45
Parsing review 46
Parsing review 47
Parsing review 48
Parsing review 49
Parsing review 50
Parsing review 51
Parsing review 52
Parsing review 53
Parsing review 54
Parsing r

/opt/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /opt/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


Parsing review 207
Parsing review 208
Parsing review 209
Parsing review 210
Parsing review 211
Parsing review 212
Parsing review 213
Parsing review 214
Parsing review 215
Parsing review 216
Parsing review 217
Parsing review 218
Parsing review 219
Parsing review 220
Parsing review 221
Parsing review 222
Parsing review 223
Parsing review 224
Parsing review 225
Parsing review 226
Parsing review 227
Parsing review 228
Parsing review 229
Parsing review 230
Parsing review 231
Parsing review 232
Parsing review 233
Parsing review 234
Parsing review 235
Parsing review 236
Parsing review 237
Parsing review 238
Parsing review 239
Parsing review 240
Parsing review 241
Parsing review 242
Parsing review 243
Parsing review 244
Parsing review 245
Parsing review 246
Parsing review 247
Parsing review 248
Parsing review 249
Parsing review 250
Parsing review 251
Parsing review 252
Parsing review 253
Parsing review 254
Parsing review 255
Parsing review 256
Parsing review 257
Parsing review 258
Parsing revi

Parsing review 805
Parsing review 806
Parsing review 807
Parsing review 808
Parsing review 809
Parsing review 810
Parsing review 811
Parsing review 812
Parsing review 813
Parsing review 814
Parsing review 815
Parsing review 816
Parsing review 817
Parsing review 818
Parsing review 819
Parsing review 820
Parsing review 821
Parsing review 822
Parsing review 823
Parsing review 824
Parsing review 825
Parsing review 826
Parsing review 827
Parsing review 828
Parsing review 829
Parsing review 830
Parsing review 831
Parsing review 832
Parsing review 833
Parsing review 834
Parsing review 835
Parsing review 836
Parsing review 837
Parsing review 838
Parsing review 839
Parsing review 840
Parsing review 841
Parsing review 842
Parsing review 843
Parsing review 844
Parsing review 845
Parsing review 846
Parsing review 847
Parsing review 848
Parsing review 849
Parsing review 850
Parsing review 851
Parsing review 852
Parsing review 853
Parsing review 854
Parsing review 855
Parsing review 856
Parsing revi

Parsing review 1389
Parsing review 1390
Parsing review 1391
Parsing review 1392
Parsing review 1393
Parsing review 1394
Parsing review 1395
Parsing review 1396
Parsing review 1397
Parsing review 1398
Parsing review 1399
Parsing review 1400
Parsing review 1401
Parsing review 1402
Parsing review 1403
Parsing review 1404
Parsing review 1405
Parsing review 1406
Parsing review 1407
Parsing review 1408
Parsing review 1409
Parsing review 1410
Parsing review 1411
Parsing review 1412
Parsing review 1413
Parsing review 1414
Parsing review 1415
Parsing review 1416
Parsing review 1417
Parsing review 1418
Parsing review 1419
Parsing review 1420
Parsing review 1421
Parsing review 1422
Parsing review 1423
Parsing review 1424
Parsing review 1425
Parsing review 1426
Parsing review 1427
Parsing review 1428
Parsing review 1429
Parsing review 1430
Parsing review 1431
Parsing review 1432
Parsing review 1433
Parsing review 1434
Parsing review 1435
Parsing review 1436
Parsing review 1437
Parsing review 1438


...

Parsing review 23079
Parsing review 23080
Parsing review 23081
Parsing review 23082
Parsing review 23083
Parsing review 23084
Parsing review 23085
Parsing review 23086
Parsing review 23087
Parsing review 23088
Parsing review 23089
Parsing review 23090
Parsing review 23091
Parsing review 23092
Parsing review 23093
Parsing review 23094
Parsing review 23095
Parsing review 23096
Parsing review 23097
Parsing review 23098
Parsing review 23099
Parsing review 23100
Parsing review 23101
Parsing review 23102
Parsing review 23103
Parsing review 23104
Parsing review 23105
Parsing review 23106
Parsing review 23107
Parsing review 23108
Parsing review 23109
Parsing review 23110
Parsing review 23111
Parsing review 23112
Parsing review 23113
Parsing review 23114
Parsing review 23115
Parsing review 23116
Parsing review 23117
Parsing review 23118
Parsing review 23119
Parsing review 23120
Parsing review 23121
Parsing review 23122
Parsing review 23123
Parsing review 23124
Parsing review 23125
Parsing revie

Parsing review 23485
Parsing review 23486
Parsing review 23487
Parsing review 23488
Parsing review 23489
Parsing review 23490
Parsing review 23491
Parsing review 23492
Parsing review 23493
Parsing review 23494
Parsing review 23495
Parsing review 23496
Parsing review 23497
Parsing review 23498
Parsing review 23499
Parsing review 23500
Parsing review 23501
Parsing review 23502
Parsing review 23503
Parsing review 23504
Parsing review 23505
Parsing review 23506
Parsing review 23507
Parsing review 23508
Parsing review 23509
Parsing review 23510
Parsing review 23511
Parsing review 23512
Parsing review 23513
Parsing review 23514
Parsing review 23515
Parsing review 23516
Parsing review 23517
Parsing review 23518
Parsing review 23519
Parsing review 23520
Parsing review 23521
Parsing review 23522
Parsing review 23523
Parsing review 23524
Parsing review 23525
Parsing review 23526
Parsing review 23527
Parsing review 23528
Parsing review 23529
Parsing review 23530
Parsing review 23531
Parsing revie

Parsing review 23883
Parsing review 23884
Parsing review 23885
Parsing review 23886
Parsing review 23887
Parsing review 23888
Parsing review 23889
Parsing review 23890
Parsing review 23891
Parsing review 23892
Parsing review 23893
Parsing review 23894
Parsing review 23895
Parsing review 23896
Parsing review 23897
Parsing review 23898
Parsing review 23899
Parsing review 23900
Parsing review 23901
Parsing review 23902
Parsing review 23903
Parsing review 23904
Parsing review 23905
Parsing review 23906
Parsing review 23907
Parsing review 23908
Parsing review 23909
Parsing review 23910
Parsing review 23911
Parsing review 23912
Parsing review 23913
Parsing review 23914
Parsing review 23915
Parsing review 23916
Parsing review 23917
Parsing review 23918
Parsing review 23919
Parsing review 23920
Parsing review 23921
Parsing review 23922
Parsing review 23923
Parsing review 23924
Parsing review 23925
Parsing review 23926
Parsing review 23927
Parsing review 23928
Parsing review 23929
Parsing revie

Parsing review 24308
Parsing review 24309
Parsing review 24310
Parsing review 24311
Parsing review 24312
Parsing review 24313
Parsing review 24314
Parsing review 24315
Parsing review 24316
Parsing review 24317
Parsing review 24318
Parsing review 24319
Parsing review 24320
Parsing review 24321
Parsing review 24322
Parsing review 24323
Parsing review 24324
Parsing review 24325
Parsing review 24326
Parsing review 24327
Parsing review 24328
Parsing review 24329
Parsing review 24330
Parsing review 24331
Parsing review 24332
Parsing review 24333
Parsing review 24334
Parsing review 24335
Parsing review 24336
Parsing review 24337
Parsing review 24338
Parsing review 24339
Parsing review 24340
Parsing review 24341
Parsing review 24342
Parsing review 24343
Parsing review 24344
Parsing review 24345
Parsing review 24346
Parsing review 24347
Parsing review 24348
Parsing review 24349
Parsing review 24350
Parsing review 24351
Parsing review 24352
Parsing review 24353
Parsing review 24354
Parsing revie

Parsing review 24890
Parsing review 24891
Parsing review 24892
Parsing review 24893
Parsing review 24894
Parsing review 24895
Parsing review 24896
Parsing review 24897
Parsing review 24898
Parsing review 24899
Parsing review 24900
Parsing review 24901
Parsing review 24902
Parsing review 24903
Parsing review 24904
Parsing review 24905
Parsing review 24906
Parsing review 24907
Parsing review 24908
Parsing review 24909
Parsing review 24910
Parsing review 24911
Parsing review 24912
Parsing review 24913
Parsing review 24914
Parsing review 24915
Parsing review 24916
Parsing review 24917
Parsing review 24918
Parsing review 24919
Parsing review 24920
Parsing review 24921
Parsing review 24922
Parsing review 24923
Parsing review 24924
Parsing review 24925
Parsing review 24926
Parsing review 24927
Parsing review 24928
Parsing review 24929
Parsing review 24930
Parsing review 24931
Parsing review 24932
Parsing review 24933
Parsing review 24934
Parsing review 24935
Parsing review 24936
Parsing revie

In [ ]:
for i, rev in enumerate(data_test.review):
    print(rev)
    test_input = test_input_data[i].copy()
    test_input = np.reshape(test_input, (1,test_input.shape[0], test_input.shape[1]))
    prediction = model.predict(test_input)
    print('Prediction: ', prediction)
    sentiment = np.argmax(prediction)
    print('Sentiment: ' + str(sentiment))